In [ ]:
import pandas as pd
import random
import json
import itertools

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, run_min_variance

In [3]:
df_all_stocks = pd.read_csv('stocks_data.csv')
cryptos_df = pd.read_csv('cryptos_data.csv')

In [4]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [5]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [6]:
# for i in range(21,25):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets) 
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)

In [19]:
#K-means with DTW
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans



def run_kmeans_dtw(df_all_stocks, n_clus=3):
    df_returns = df_all_stocks.pct_change().dropna()
    data_kmeans = df_returns.T.values

    tickers = list(df_all_stocks.columns)

    data_scaled = TimeSeriesScalerMeanVariance().fit_transform(data_kmeans)

    model = TimeSeriesKMeans(n_clusters=n_clus, metric="dtw", random_state=0)
    labels = model.fit_predict(data_scaled)

    tickers_with_lables = {k: int(v) for k, v in zip(tickers, labels)}

    return tickers_with_lables

In [18]:
run_kmeans_dtw(df_all_stocks, n_clus=3)

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


{'AAPL': 1,
 'ADBE': 1,
 'AMD': 1,
 'AMZN': 1,
 'ASML': 1,
 'AVGO': 0,
 'AZN': 1,
 'COST': 1,
 'CSCO': 2,
 'ERIC': 1,
 'GOOG': 1,
 'ISRG': 1,
 'LIN': 1,
 'META': 0,
 'MSFT': 1,
 'NFLX': 1,
 'PEP': 1,
 'TMUS': 1,
 'TSLA': 1,
 'ASX': 1,
 'BRK-B': 1,
 'EC': 1,
 'HDB': 1,
 'IBN': 1,
 'JPM': 1,
 'LLY': 0,
 'NMR': 1,
 'NVO': 0,
 'ORCL': 1,
 'PBR': 1,
 'PKX': 0,
 'RDY': 1,
 'SONY': 1,
 'TSM': 0,
 'UMC': 0,
 'UNH': 1,
 'V': 1,
 'WIT': 1,
 'XOM': 1,
 'AAOI': 0,
 'ADMA': 0,
 'BITF': 0,
 'CRSP': 1,
 'DJT': 0,
 'ECX': 0,
 'ENPH': 1,
 'ETNB': 0,
 'INOD': 0,
 'IOVA': 0,
 'KROS': 1,
 'MNKD': 1,
 'OXLC': 1,
 'PAYO': 1,
 'PRCT': 1,
 'PSNY': 0,
 'RZLV': 0,
 'SMMT': 0,
 'TTD': 0,
 'AI.PA': 1,
 'AIR.PA': 1,
 'BNP.PA': 1,
 'CS.PA': 1,
 'DG.PA': 1,
 'DSY.PA': 0,
 'EL.PA': 1,
 'MC.PA': 1,
 'OR.PA': 1,
 'SAF.PA': 1,
 'SAN.PA': 2,
 'SGO.PA': 1,
 'SU.PA': 1,
 'TTE.PA': 1,
 '0DH.F': 0,
 '3YU.F': 0,
 '8GC.F': 1,
 'BPE5.F': 1,
 'CBA.F': 2,
 'CNE.F': 0,
 'IUQ.F': 1,
 'KM6.F': 1,
 'LFU2.F': 0,
 'LPS1.F': 0,
 'NISA.F